# Dataset clean

## Generic functions

In [ ]:
import re
def verbosity(msj, is_verbosity):
  if is_verbosity:
    print(msj)

def search_str_in_df(df_link, df_col, str_):
  return [(link, farm_size) for farm_size, link in zip(df_col, df_link) if type(farm_size)==str and str_ in farm_size]

## load file

In [ ]:
import pandas as pd

file_path = '../data/AuctionsV2.xlsx'

try:
  global df
  df = pd.read_excel(file_path)
except FileNotFoundError:
  print(f"Error: File not found at {file_path}")
except Exception as e:
  print(f"An error occurred: {e}")




In [ ]:
df.head(1)

In [ ]:
df.describe()

In [ ]:
# Order values counts by year
df['Year'].value_counts().sort_index()

In [ ]:
df['Country'].value_counts()

In [ ]:

df['Farm_Size'].value_counts().head(20)

In [ ]:
# Remove "ha" from the 'Farm_Size' column
df['Farm_Size'] = df['Farm_Size'].str.replace('ha', '', case=False, regex=False)

In [ ]:
# df[df['Farm_Size'].str.contains("hectares")]

In [ ]:
df['Farm_Size'] = df['Farm_Size'].str.replace('hectares farm in production', '', case=False)

In [ ]:
df['Farm_Size'] = df['Farm_Size'].str.replace('-?hec.*', '', case=False, regex=True)

In [ ]:
df.loc[[4678, 4675]]

## remove all totals rows

In [ ]:

def remove_rows_if_condition(condition, index_):
    global df
    if condition:
        print(f"drop {index_} row")
        df.drop(index=index_, inplace=True)

[remove_rows_if_condition(("totals:" in str(value_) or "total" in str(value_) or "stat" in str(value_)), index_) for value_, index_ in zip(df['Rank'], df.index)]
df.reset_index(drop=True, inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)



In [ ]:
is_verbosity = False

# Function to clean and convert farm size
def clean_farm_size(link, size, index_):
  if size == "-":
    size = None

  if pd.isna(size):
      return size  # Return NaN values as they are

  size = str(size)
  
  if size == '.':
    return None
  
  if '()' in size:
    size = size.replace('()', '')
  if ' .' in size:
    size = size.replace(' .', '')

  if '/' in size:
    size = size.split('/')[1]
  
  if "m2" in size or "m²" in size or "metros cuadrados" in size:
    size = size.replace("m2", "").replace("m²", "").replace("metros cuadrados", "")
    aux_size = size
    size = size.replace(",", ".")
    verbosity(f"size m2 = {size}", is_verbosity)

    try:
      if float(size) <100:
        size = aux_size.replace(",", "")

      verbosity(f"Successfull changed size to ha = {float(size) / 10000}", is_verbosity)
      return float(size) / 10000
    except ValueError:
      verbosity(f"Error: {ValueError} in size: {size}", is_verbosity)
      return size # Return original string if conversion fails.
  
  if "acres" in size:
    size = size.replace("acres", "")
    aux_size = size
    size = size.replace(",", ".")
    verbosity(f"size acres = {size}", is_verbosity)

    try:
      verbosity(f"Successfull changed size to ha = {float(size) * 0.404686}", is_verbosity)
      return float(size) * 0.404686
    except ValueError:
      verbosity(f"Error: {ValueError} in size: {size}", is_verbosity)
      return size # Return original string if conversion fails.

  if "patok" in size:
    size = size.replace("patok", "")
    aux_size = size
    size = size.replace(",", ".")
    verbosity(f"size patok = {size}", is_verbosity)

    try:
      verbosity(f"Successfull changed size to ha = {float(size) * 0.1}", is_verbosity)
      return float(size) * 0.1
    except ValueError:
      verbosity(f"Error: {ValueError} in size: {size}", is_verbosity)
      return size # Return original string if conversion fails.
    
  

  if "MANZANAS" in size or "mzns" in size or "manzanas" in size or "mz" in size:
    size = size.replace("manzanas", "").replace("mzns", "").replace("MANZANAS", "").replace(",", "").replace("mz", "")
    verbosity(f"size mzn = {size}", is_verbosity)
    try:
      verbosity(f"Successfull changed size to ha = {float(size) * 0.698896}", is_verbosity)
      return float(size) * 0.698896
    except ValueError:
      if "y media" in size:
        size = size.replace("y media", "")
        try:
          verbosity(f"Successfull changed size to ha = {(float(size)+0.5) * 0.698896}", is_verbosity)
          return (float(size)+0.5) * 0.698896
        except ValueError as e:
          verbosity(f"Error: {e} in size: {size}", is_verbosity)
          return size # Return original string if conversion fails.
      verbosity(f"Error: {ValueError} in size: {size}", is_verbosity)
      return size # Return original string if conversion fails.

  try:
    float(size)
  except ValueError as e:
    if "," in size:
      try:
        size = size.replace(",", ".")
        return size
      except:
        print(f"Error: {e} in size: {size}")
        return size

    print(f"Second Error: {e} in size: {size}, index: {index_}, link {link}")
    return size

  return size

# [(link, value_) for value_, link in zip(df_col, df_link) if not(pd.isna(value_)) and type(float(value_))==float]
# Apply the function to the 'Farm_Size' column

df['Farm_size_he'] = pd.DataFrame( [clean_farm_size(link, value_, index_) for value_, link, index_ in zip(df['Farm_Size'], df['Url_farm'], df.index)])
# df['farm_size_he'] = df['Farm_Size'].apply(clean_farm_size, link=df['Url_farm'])




In [ ]:
df.describe(include='all')

In [ ]:
df.head()

In [ ]:
def val_float_in_df(df_link, df_col):
  return [(link, value_) for value_, link in zip(df_col, df_link) if not(pd.isna(value_)) and type(float(value_))==float]
search_str_in_df(df['Url_farm'], df['Farm_size_he'], "m")
# val_float_in_df(df['Url_farm'], df['Farm_size_he'])

In [ ]:
df.to_excel('../data/AuctionsV4.xlsx')

In [ ]:
#Revisar caso row 4301
df.loc[[4289, 4298,4301]]

## Generic functions

In [ ]:

def is_float_data_detection(link, value_, index_, verbosity):
    if pd.isna(value_):
        return True

    value_ = str(value_)

    try:
        float(value_)
        return True
    except:
        if verbosity:
            print(f"detected non float data {value_} in row {index_}; more information in {link}")
        return False

def non_float_data_detection(col_):
    [is_float_data_detection(link, value_, index_, True) for value_, link, index_ in zip(df[col_], df['Url_farm'], df.index)]

def str_2_float(value_):
    try:
        value_ = float(value_)
        return True, value_
    except:
        return False, value_


def apply_clean_function_to_data(col_, clean_function):
    global df
    return pd.DataFrame( [clean_function(link, value_, index_) for value_, link, index_ in zip(df[col_], df['Url_farm'], df.index)])

def count_number_of_digits(value_):
    if type(value_) is float:
        value_ = str(value_)
    
    number_of_digits = len(value_)
    if '.' in value_:
        number_of_digits = number_of_digits - 3
    
    return number_of_digits
    
def replace_value(value_, str_replace, replace_for):
    if str_replace in value_:
        value_ = value_.replace(str_replace, replace_for)
    return value_

def detect_abnormal_commas(value_, is_float):

    if not(is_float):
        patron = re.compile('\d{1,3}.\d\d\d,\d\d') # coincide con una letra, seguida de al menos 1 dígito entre 3 y 5
        result = patron.search(value_)

        if(result):
            value_ = result.group()
            value_ = value_.replace('.', '')
            value_ = value_.replace(',', '.')
            is_float, value_ = str_2_float(value_)
            return is_float, value_

        patron = re.compile('\d{1,3},\d\d\d\.\d\d') # coincide con una letra, seguida de al menos 1 dígito entre 3 y 5
        result = patron.search(value_)
        if(result):
            value_ = result.group()
            value_ = value_.replace(',', '')
            is_float, value_ = str_2_float(value_)
            return is_float, value_
        
        patron = re.compile('\d{1,3},\d{1,3}') # coincide con una letra, seguida de al menos 1 dígito entre 3 y 5
        result = patron.search(value_)
        if(result):
            value_ = result.group()
            value_ = value_.replace(',', '.')
            is_float, value_ = str_2_float(value_)

    return is_float, value_

def units_conversion(value_, conversion_to_multiplly, unit):
    value_ = value_.replace(value_, unit)

    try:
        value_ = float(value_)
        return (value_ * conversion_to_multiplly)
        print(f"se convirtió el valor a {unit} exitosamente")
    except ValueError:
        return value_

In [ ]:

cadena = '$20.734,26'
patron = re.compile('\d{1,3}\.\d\d\d,\d\d') # coincide con una letra, seguida de al menos 1 dígito entre 3 y 5
result = patron.search(cadena)
# result = patron.findall(cadena)
if(result):
    value_ = result.group()
    value_ = value_.replace('.', '')
    value_ = value_.replace(',', '.')
    print(value_)


In [ ]:

cadena = '62,70'

patron = re.compile('\d{1,3},\d{1,3}') # coincide con una letra, seguida de al menos 1 dígito entre 3 y 5
result = patron.search(cadena)
# result = patron.findall(cadena)
if(result):
    value_ = result.group()
    value_ = value_.replace(',', '.')
    print(value_)


## clean score lots in table

In [ ]:
def clean_score_lots(link, value_, index_):
    is_float = is_float_data_detection(link, value_, index_, False)
    if is_float:
        return  float(value_)
    if (',' in value_) and (not(is_float)):
        value_ = value_.replace(',', '.')
        is_float, value_ = str_2_float(value_)   
    
    if is_float:
        return  float(value_)
    else:
        print(f"detected not float data {value_} in row {index_}; more information in {link}")
        return value_


df['Score_v2'] = apply_clean_function_to_data('Score', clean_score_lots)
# non_float_data_detection('Score_v2')

## clean Total Value

In [ ]:
def clean_total_value(link, value_, index_):
    global df
    value_ = str(value_)
    aux_value_2 = value_
    value_ = replace_value(value_, '$', '')
    value_ = replace_value(value_, '/lb', '')
    value_ = replace_value(value_, '/Ib', '')
    value_ = replace_value(value_, 'US', '')
    value_ = replace_value(value_, '.              ', '')

    is_float = is_float_data_detection(link, value_, index_, False)
    
    if is_float:
        return  float(value_)
    
    if ('-' in value_) and (not(is_float)):
        value_ = None
        return value_
    if ('–' in value_) and (not(is_float)):
        value_ = None
        return value_
    
    is_float, value_ = detect_abnormal_commas(value_, is_float)

    if is_float:
        if count_number_of_digits(value_) > 3:
            # TODO: divide high_bid / weight lb
            # print(f"detected abnormal number: weight lb: {df.loc()[index_]['Weigth_(Lb)']}; weight kg: {df.loc()[index_]['Weigth_(Kg)']};initial value: {aux_value_2}; processed value {value_} in row {index_}; more information in {link}")
            pass

        return  float(value_)
    else:
        print(f"detected not float data {value_} in row {index_}; more information in {link}")
        return value_

df['Total_Value_v2'] = apply_clean_function_to_data('Total_Value', clean_total_value)

    

## clean weight in table

In [ ]:
def clean_weight(link, value_, index_):
    global df

    value_lb = value_
    value_kg = df.loc[index_]['Weigth_(Kg)']
    if value_lb:
        value_ = value_lb
    else:
        value_ = value_kg
        units_conversion(value_, 2.20462, 'lbs')
        
    value_ = str(value_)
    aux_value_2 = value_

    is_float = is_float_data_detection(link, value_, index_, False)
    
    if is_float:
        return  float(value_)
    
    if ('-' in value_) and (not(is_float)):
        value_ = None
        return value_
    
    is_float, value_ = detect_abnormal_commas(value_, is_float)

    if is_float:
        if count_number_of_digits(value_) > 4:
            print(f"detected abnormal number: weight lb: {df.loc()[index_]['Weigth_(Lb)']}; weight kg: {df.loc()[index_]['Weigth_(Kg)']};initial value: {aux_value_2}; processed value {value_} in row {index_}; more information in {link}")
            pass

        return  float(value_)
    else:
        print(f"detected not float data {value_} in row {index_}; more information in {link}")
        return value_

df['Weigth_(Lb)_v2'] = apply_clean_function_to_data('Weigth_(Lb)', clean_weight)
# non_float_data_detection('Score_v2')

## clean Size 30Kg boxes

In [ ]:
def clean_high_bid(link, value_, index_):
    global df
    value_ = str(value_)
    aux_value_2 = value_
    # value_ = replace_value(value_, '$', '')
    # value_ = replace_value(value_, '/lb', '')
    # value_ = replace_value(value_, '/Ib', '')
    # value_ = replace_value(value_, 'US', '')

    is_float = is_float_data_detection(link, value_, index_, False)
    
    if is_float:
        return  float(value_)
    
    if ('-' in value_) and (not(is_float)):
        value_ = None
        return value_
    
    is_float, value_ = detect_abnormal_commas(value_, is_float)

    if is_float:
        if count_number_of_digits(value_) > 2:
            # TODO: divide high_bid / weight lb
            print(f"SE RECOMIENDA ELIMINAR ESTAS FILAS AL FINAL detected abnormal number: Size 30Kg boxes: {df.loc()[index_]['Size_30Kg_boxes']}; weight kg: {df.loc()[index_]['Weigth_(Kg)']};initial value: {aux_value_2}; processed value {value_} in row {index_}; more information in {link}")

        return  float(value_)
    else:
        print(f"detected not float data {value_} in row {index_}; more information in {link}")
        return value_

df['Size_30Kg_boxes_v2'] = apply_clean_function_to_data('Size_30Kg_boxes', clean_high_bid)
# non_float_data_detection('Score_v2')



## complete weight Lb

In [ ]:
def complete_weight_lb(link, value_, index_):
    global df
    weight_lb = df.loc[index_]['Weigth_(Lb)_v2']

    size_30kg_boxes_is_float = is_float_data_detection(link, value_, index_, False)

    if size_30kg_boxes_is_float:
        size_30kg_boxes = float(value_)
    else: 
        return

    if pd.isna(weight_lb):
        weight_lb = (size_30kg_boxes*132.277)/2
        print(f"used size_30Kb boxes: {value_} to complete weight_(lb): {weight_lb} in row {index_}; more information in {link}")
    
    return weight_lb

df['Weigth_(Lb)_v2'S] = apply_clean_function_to_data('Size_30Kg_boxes_v2', complete_weight_lb)
pass




## clean high bid in table

In [ ]:
def clean_high_bid(link, value_, index_):
    global df
    value_ = str(value_)
    aux_value_2 = value_
    value_ = replace_value(value_, '$', '')
    value_ = replace_value(value_, '/lb', '')
    value_ = replace_value(value_, '/Ib', '')
    value_ = replace_value(value_, 'US', '')

    is_float = is_float_data_detection(link, value_, index_, False)
    
    if is_float:
        return  float(value_)
    
    if ('-' in value_) and (not(is_float)):
        value_ = None
        return value_
    
    is_float, value_ = detect_abnormal_commas(value_, is_float)

    if is_float:
        if count_number_of_digits(value_) > 3:
            # TODO: divide high_bid / weight lb
            # print(f"detected abnormal number: weight lb: {df.loc()[index_]['Weigth_(Lb)']}; weight kg: {df.loc()[index_]['Weigth_(Kg)']};initial value: {aux_value_2}; processed value {value_} in row {index_}; more information in {link}")
            pass

        return  float(value_)
    else:
        print(f"detected not float data {value_} in row {index_}; more information in {link}")
        return value_

df['High_Bid_v2'] = apply_clean_function_to_data('High_Bid', clean_high_bid)
# non_float_data_detection('Score_v2')

## detect abnormal data

In [ ]:
def look_for_abnormal_data(link, value_, index_):

    if pd.isna(value_):
        return

    weight_lb = value_
    high_bid = df.loc[index_]['High_Bid_v2']
    total_value = df.loc[index_]['Total_Value_v2']

    aux_weight_lb = total_value/high_bid
    aux_weight_lb = round(aux_weight_lb, 2)
    difference = aux_weight_lb-weight_lb
    if(difference>1):
        print(f"detected inconsistences in row {index_}; more information in {link}")
     


apply_clean_function_to_data('Weigth_(Lb)_v2', look_for_abnormal_data)
pass

## save all data

In [ ]:
# df.drop(columns=['Weigth_(Kg)'], inplace=True)
# df.drop(columns=['Weigth_(Lb)'], inplace=True)
# df.drop(columns=['High_Bid'], inplace=True)


df.to_excel("../data/AuctionsV5.xlsx", engine='xlsxwriter')
